In [1]:
__PRODUCTION__ = 1
__NAME__       = 'image'
__WIDTH__      = 5.5
__HEIGHT__     = 3.2

if __PRODUCTION__:
    from mplmagic2 import pgf
else:
    from mplmagic2 import svg

from mplmagic2 import SuperFigure, tex
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patheffects as path_effects

print('This is how much space the figure will take up on letter paper')
SuperFigure.size_hint(__WIDTH__, __HEIGHT__, margin_left=0.5 * (8.5 - __WIDTH__));

This is how much space the figure will take up on letter paper


In [2]:
import matplotlib.pyplot as plt
import functools
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.distance import cdist
import torch
import torch.nn.functional as F
from torch import optim
import pickle

from symfac.experimental import RBFExpansionPlus
from symfac.experimental import RBFExpansionMiniBatchPlus

In [3]:
fig = SuperFigure(figsize=(__WIDTH__, __HEIGHT__), dpi=300)
ax_canvas = fig.make_axes(
    left=0, right=1, top=0, bottom=1, zorder=-100,
    style='blank' if __PRODUCTION__ else None
)
ax_canvas.set_xlim([0, 1])
ax_canvas.set_ylim([0, 1])

# draw the primary axes grid
x0 = 0.005
y0 = 0.04
dx = 0.51
w = 0.48
h = 0.4
axs_anchor = [
    fig.make_axes(
        left=x0 + dx * i,
        top=y0,
        width=w,
        height=h,
        style='blank'
    ) for i in range(2)
]

# draw the primary compare grid
axs_1st = []
for ax in axs_anchor:
    ncol = 2
    wspace = 0.02
    
    w2 = (ax.right - ax.left - wspace * (ncol - 1)) / ncol
    axs_1st.append([
        fig.make_axes(
            left=ax.x0 + j * (w2 + wspace),
            top=ax.y0,
            width=w2,
            # height=h2,
            width_to_height=1,
            style='modern'
        )
        for j in range(ncol)
    ])
axs_1st = np.array(axs_1st)

# draw the secondary zoom-in grid
axs_2nd = []
for ax in axs_anchor:
    nrow = 3
    ncol = 5
    hspace = 0.04
    wspace = 0.025
    
    w2 = (ax.right - ax.left - wspace * (ncol - 1)) / ncol
    h2 = (ax.bottom - ax.top - hspace * (nrow - 1)) / nrow
    axs_2nd.append([
        [
            fig.make_axes(
                left=ax.x0 + j * (w2 + wspace),
                top=ax.y1 + i * (h2 + hspace) + 0.13,
                width=w2,
                # height=h2,
                width_to_height=1,
                style='modern'
            )
            for j in range(ncol)
        ] for i in range(nrow)
    ])
axs_2nd = np.array(axs_2nd)

# # draw the rank-r compares
# hspace = 0.05
# h = (axs_2nd[0][0].height - hspace) / 2
# # hspace = 
# axs_3rd = [
#     [
#         [
#             fig.make_axes(
#                 left=ax2.x1 + i * 0.06 + 0.04,
#                 top=ax2.y0 + j * (h + hspace),
#                 height=h,
#                 width_to_height=1.0,
#                 style='modern'
#             ) for i in range(4) for j in range(2)
#         ]
#         for ax2 in axs2
#     ]
#     for axs2 in axs_2nd
# ]

def SVD_recon(U, S, V, r):
    return (U[:, :r] * S[:r]) @ V[:r, :]

image_style = dict(
    cmap='Greys_r'
)

windows = dict(
    poster=[
        (12, 36),
        (48, 196),
        (204, 120),
    ],
    baboon=[
        (172, 99),
        (12, 144),
        (197, 156),
    ]
)

title_style = dict(
    fontsize=8,
    y=0.95,
    va='bottom'
)

overview_rank = 32
zoomin_ranks = [32, 64]
for axs1, axs2, (iname, fname) in zip(
    axs_1st,
    axs_2nd,
    [('poster', 'data/paper/space-tourists-poster-256.png'),
     ('baboon', 'data/paper/baboon-256x256.png')]
):
    target = plt.imread(fname)
    # RBG to grayscale
    target = target[:, :, 0] * 0.299 + target[:, :, 1] * 0.587 + target[:, :, 2] * 0.114
    n, m = target.shape
    U, S, V = np.linalg.svd(target)
    axs1[0].imshow(SVD_recon(U, S, V, overview_rank), **image_style)
    fac = RBFExpansionMiniBatchPlus.from_pickle(
        f'data/paper/{iname}-{overview_rank:03d}-component-lr0.025-mini8x.pickle'
    )
    best_run = np.argmin(fac.report.loss_best)
    fac_recon = fac.optimum(runs=best_run)
    axs1[1].imshow(fac_recon, **image_style)
    
    axs1[0].set_title(fr'\textbf{{SVD {overview_rank} Ranks}}', **title_style)
    axs1[1].set_title(fr'\textbf{{RBF {overview_rank} Components}}', **title_style)
    
    ww = 40
    axs2[0, 0].set_title(r'Original', **title_style)
    for k, r in enumerate(zoomin_ranks):
        ax_bg = fig.make_axes(
            left=axs2[0, k * 2 + 1].left, right=axs2[0, k * 2 + 2].right,
            top=axs1[0].bottom, bottom=axs2[0, 1].top, style='blank'
        )
        ax_bg.set_xlim([0, 1])
        ax_bg.set_ylim([0, 1])
        ax_bg.text(
            0.5, 0.55,
            f'{r} Components',
            va='bottom',
            ha='center',
            fontsize=8
        )
        ax_bg.axhline(0.475, lw=0.5, color='k', ls=(2, (1, 3)), dash_capstyle='round')
    
    title_style_mini = dict(
        fontsize=6,
        y=0.825,
        va='bottom',
        ha='center',
        linespacing=1.0,
    )
    for j, r in enumerate(zoomin_ranks):
        fac = RBFExpansionMiniBatchPlus.from_pickle(
            f'data/paper/{iname}-{r:03d}-component-lr0.025-mini8x.pickle'
        )
        svd_loss = F.mse_loss(torch.tensor(SVD_recon(U, S, V, r)), torch.tensor(target))
        axs2[0, j * 2 + 1].set_title(
            f'SVD Loss\n${tex.sciform(svd_loss, frac_digits=1, exp_digits=1)}$',
            **title_style_mini
        )
        axs2[0, j * 2 + 2].set_title(
            f'RBF Loss\n${tex.sciform(fac.report.loss_best.min(), frac_digits=1, exp_digits=1)}$',
            **title_style_mini
        )
#     axs2[0, 1].set_title(f'SVD Loss {}', **title_style_mini)
#     axs2[0, 2].set_title(f'RBF Loss {}', **title_style_mini)
#     axs2[0, 3].set_title('SVD', **title_style_mini)
#     axs2[0, 4].set_title('RBF', **title_style_mini)
    for k, (wx, wy) in enumerate(windows[iname]):
        rect_style = dict(
            facecolor='none',
            edgecolor='#FF6060',
            lw=1.0,
        )
        text_style = dict(
            fontsize=8,
            color='k',
            va='center',
            ha='center',
            path_effects=[
                path_effects.Stroke(linewidth=2.0, foreground='w'),
                path_effects.Normal()
            ]
        )
        for ax in axs1:
            ax.add_patch(Rectangle((wy, wx), ww, ww, **rect_style))
            # ax.add_patch(Rectangle((wy, wx), ww, ww, edgecolor='k', ls=(2, (1, 1)), **rect_style))
            # ax.add_patch(Rectangle((wy, wx), ww, ww, edgecolor='w', ls=(1, (1, 1)), **rect_style))
            ax.text(
                wy + 0.5 * ww, wx + 0.5 * ww,
                f'{k + 1}', **text_style
            )
        axs2[k][0].imshow(target[wx:wx + ww, wy:wy + ww], **image_style)
        for j, r in enumerate(zoomin_ranks):
            fac = RBFExpansionMiniBatchPlus.from_pickle(
                f'data/paper/{iname}-{r:03d}-component-lr0.025-mini8x.pickle'
            )
            best_run = np.argmin(fac.report.loss_best)
            fac_recon = fac.optimum(runs=best_run)
            axs2[k][j * 2 + 1].imshow(SVD_recon(U, S, V, r)[wx:wx + ww, wy:wy + ww], **image_style)
            axs2[k][j * 2 + 2].imshow(fac_recon[wx:wx + ww, wy:wy + ww], **image_style)


def fix_axes(ax):
    ax.set_xticks([])
    ax.set_yticks([])

for ax in axs_1st.ravel():
    fix_axes(ax)
for ax in axs_2nd.ravel():
    fix_axes(ax)


# axs[0].set_title(r'\textbf{Gradient Descent}', **title_style)
# axs[1].set_title(r'\textbf{Stochastic Gradient Descent}', **title_style)
# axs[2].set_title(r'\textbf{Histogram of Final Loss}', **title_style)


# tick_style = dict(
#     fontsize=7
# )
# for ax in axs[:3]:
#     ax.patch.set_facecolor('w')
#     ax.set_xbound(lower=0, upper=K.shape[0])
#     ax.set_ybound(lower=0, upper=K.shape[1])
#     xticks = np.array([0, K.shape[0]])
#     ax.set_xticks(xticks)
#     ax.set_yticks([])
#     ax.tick_params(axis='both', which='both', length=0)
#     ax.set_xticklabels(['%d' % x for x in xticks], **tick_style)
#     # ax.set_yticklabels(['%d' % y for y in K.shape[1] - yticks], **tick_style)
#     ax.set_xlabel(r'\textbf{Node ID}', fontsize=8, va='bottom')

# starts = np.concatenate(([0], np.cumsum(sizes)))
# u = fac.optimum.u[best_run, :, 0].cpu().numpy()
# u = (u - u.min()) / (u.max() - u.min())  # normalize to 0-1
# for i, (beg, end) in enumerate(zip(starts[:-1], starts[1:])):
#     axs[3].scatter(
#         np.arange(beg, end),
#         u[beg:end],
#         s=8.0,
#         marker='o',
#         color=colorsys.hls_to_rgb(i * 0.125, 0.45, 1.0),
#         # edgecolor='w',
#         edgecolor=colorsys.hls_to_rgb(i * 0.125, 0.3, 1.0),
#         linewidth=.5
#     )
# axs[3].set_ylabel(r'\textbf{Embedded Coordinate}', fontsize=8)
# axs[3].set_xbound(lower=-4, upper=K.shape[0] + 4)
# axs[3].set_ybound(lower=-0.2, upper=1.2)
# xticks = np.array([0, K.shape[0]])
# axs[3].set_xticks(xticks)
# axs[3].set_yticks([0, 1])
# axs[3].tick_params(axis='both', which='both', length=0)
# axs[3].set_xticklabels(['%d' % x for x in xticks], **tick_style)
# axs[3].set_yticklabels([0, 1], **tick_style)
# axs[3].set_xlabel(r'\textbf{Node ID}', fontsize=8, va='bottom')

# title_style = dict(
#     fontsize=8,
#     y=0.975,
#     va='bottom'
# )
# axs[0].set_title(r'\textbf{Original Adjacency Matrix}', **title_style)
# axs[1].set_title(r'\textbf{One SVD Rank}', **title_style)
# axs[2].set_title(r'\textbf{One RBF Component}', **title_style)
# axs[3].set_title(r'\textbf{Normalized RBF Embedding}', **title_style)


if __PRODUCTION__:
    fig.savefig(f'pgf/{__NAME__}.pgf', dpi=300)
else:
    fig.savefig(f'svg/{__NAME__}.svg', dpi=300)
plt.show()

In [4]:
!make -f Makefile.figures fig-"$__NAME__".pdf 2>&1 | tail -n 1

Successfully created fig-image.pdf
